In [1]:
import numpy as np
import torch
from torch import nn
from torch import optim

from models import SelectiveKernel, SKCNNBlock, SKCNN
from preprocess import preprocess

In [2]:
# Data parameters
W = 50000 # Max length of input vector
C_in = 1 # Input number of channels
num_classes = 11 # Number of radar emitter types
sk_block_params = [ # Selective Kernel block parameters
{
    'C_out' : 16,
    'kernels' : [9, 16],
    'fc_shared_sizes' : [16],
    'fc_indep_sizes' : [],
    'activation_fn' : nn.ReLU,
    'pooling_size' : 7,
    'stride' : 7
},
{
    'C_out' : 32,
    'kernels' : [9, 16],
    'fc_shared_sizes' : [32],
    'fc_indep_sizes' : [],
    'activation_fn' : nn.ReLU,
    'pooling_size' : 7,
    'stride' : 7
},
{
    'C_out' : 64,
    'kernels' : [9, 16],
    'fc_shared_sizes' : [64],
    'fc_indep_sizes' : [],
    'activation_fn' : nn.ReLU,
    'pooling_size' : 7,
    'stride' : 7
},
{
    'C_out' : 128,
    'kernels' : [9, 16],
    'fc_shared_sizes' : [128],
    'fc_indep_sizes' : [],
    'activation_fn' : nn.ReLU,
    'pooling_size' : 7,
    'stride' : 7
}
]
fc_block_sizes=[512] # Fully Connected Layer parameters

In [3]:
# Example input - Replace with respective data processing pipeline
x = torch.rand(1, C_in, W)

# 1-D Convolutional Neural Networks for Intrapulse Modulation

The 1D CNN network for classification of intrapulse signals follows from the work in \url[Intra-Pulse Modulation Classification of Radar Emitter Signals Based on a 1-D Selective Kernel Convolutional Neural Network](https://www.mdpi.com/2072-4292/13/14/2799/htm) is explored in this notebook. Following the author's work, the network consists of 4 blocks that comprises of a Selective Kernel Convolutional layer, a max-pooling layer and a batch normalisation layer. 

In [4]:
# Initialize SKCNN network
skcnn = SKCNN(
    W, 
    C_in,
    sk_block_params=sk_block_params,
    fc_block_sizes=fc_block_sizes)
skcnn

SKCNN(
  (skcnn_blocks): Sequential(
    (0): SKCNNBlock(
      (selective_kernel): SelectiveKernel(
        (activation_fn): ReLU()
        (softmax): Softmax(dim=0)
        (conv_layers): ModuleList(
          (0): Conv1d(1, 16, kernel_size=(9,), stride=(1,), padding=same)
          (1): Conv1d(1, 16, kernel_size=(16,), stride=(1,), padding=same)
        )
        (mlp): ModuleList(
          (0): Linear(in_features=16, out_features=16, bias=True)
        )
        (output_heads): ModuleList(
          (0): Sequential(
            (0): Linear(in_features=16, out_features=16, bias=True)
          )
          (1): Sequential(
            (0): Linear(in_features=16, out_features=16, bias=True)
          )
        )
      )
      (max_pooling): MaxPool1d(kernel_size=7, stride=7, padding=0, dilation=1, ceil_mode=False)
      (batch_norm): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): SKCNNBlock(
      (selective_kernel): SelectiveKernel(
  

### Preprocessing

The preprocessing step for input signals is simple and consists of only three steps:
1. Zero padding of input signals to ensure fixed length vectors required for the CNN.
2. Fast Fourier Transform to convert the signal into frequency domain
3. Amplitude normalization to remove effects of large amplitude values on classification

In [5]:
# Preprocess data
from preprocess import *
import torch

x_preprocessed = preprocess(x)
x_preprocessed = torch.tensor(x_preprocessed)

In [6]:
skcnn(x_preprocessed)

/home/lukaz/.local/lib/python3.8/site-packages/torch/nn/modules/conv.py:294: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  /pytorch/aten/src/ATen/native/Convolution.cpp:660.)
  return F.conv1d(input, weight, bias, self.stride,
/home/lukaz/.local/lib/python3.8/site-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)
/home/lukaz/.local/lib/python3.8/site-packages/torch/nn/modules/container.py:139: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


tensor([[0.0697, 0.1279, 0.0617, 0.1129, 0.1290, 0.0508, 0.0687, 0.1063, 0.1163,
         0.1066, 0.0502]], grad_fn=<SoftmaxBackward>)